In [1]:

from pymongo import MongoClient, ASCENDING, DESCENDING, HASHED
from tqdm.notebook import tqdm
import numpy as np
import torch



In [2]:
conn = MongoClient()

process_fibvid = conn.process_fibvid
tweets_tokens  = process_fibvid.tweets_tokens
tweets_bert  = process_fibvid.tweets_bert
tweets_bert_fast  = process_fibvid.tweets_bert_fast

tweets_bert_fast.create_index([('num_tokens', ASCENDING)])
tweets_bert_fast.create_index([('created', ASCENDING)])
tweets_bert_fast.create_index([('tweetId', HASHED)])
tweets_bert_fast.create_index([('userId', HASHED)])

'userId_hashed'

In [3]:
total = tweets_tokens.count_documents({})

DB_BATCH_SIZE = 500
db_batch = []

with conn.start_session() as session: 
    for tweet in tqdm(tweets_bert.find(no_cursor_timeout=True, 
                                        session=session).sort([('num_tokens', DESCENDING)]), total=total):
        del tweet['last_hidden_state']
        db_batch.append(tweet)
        if len(db_batch) > DB_BATCH_SIZE:
            tweets_bert_fast.insert_many(db_batch)
            db_batch = []


if len(db_batch) > 0:
    tweets_bert_fast.insert_many(db_batch)
    db_batch = []

  0%|          | 0/299118 [00:00<?, ?it/s]

In [4]:
#db_batch[0].keys()